# Text Scam Detection - Milestone I

This notebook implements the Core Text Detector module for detecting digital arrest & fraud scams in text communications.

Features:
- TF-IDF + Logistic Regression baseline
- Fine-tuned DistilRoBERTa transformer
- Rule-based keyword detection
- Explainability with SHAP
- FastAPI microservice

Dataset: SMS Spam Collection + synthetic scam/legit texts

In [ ]:
# Install dependencies
!pip install scikit-learn transformers datasets joblib shap fastapi uvicorn pandas torch pyngrok

In [ ]:
import os
import pandas as pd
import requests
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score
import joblib
import logging
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset as HFDataset
import shap
import yaml
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import nest_asyncio

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Data Ingestion and Preparation

In [ ]:
DATA_DIR = "data/text"
MANIFEST_PATH = "data/text_manifest.csv"
os.makedirs(DATA_DIR, exist_ok=True)

def download_file(url, dest_path):
    logger.info(f"Downloading {url} to {dest_path}")
    response = requests.get(url)
    response.raise_for_status()
    with open(dest_path, 'wb') as f:
        f.write(response.content)

def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def load_sms_spam():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
    zip_path = os.path.join(DATA_DIR, "smsspamcollection.zip")
    extract_to = os.path.join(DATA_DIR, "smsspam")

    if not os.path.exists(extract_to):
        download_file(url, zip_path)
        extract_zip(zip_path, extract_to)

    file_path = os.path.join(extract_to, "SMSSpamCollection")
    df = pd.read_csv(file_path, sep='\t', header=None, names=['label', 'text'])
    df['label'] = df['label'].map({'ham': 0, 'spam': 1})
    df['source'] = 'sms_spam'
    return df

def load_enron_ham():
    ham_texts = [
        "Meeting scheduled for tomorrow at 10 AM.",
        "Please find attached the quarterly report.",
        "Thank you for your email. I will review it.",
        "Reminder: Team lunch on Friday.",
        "Invoice attached for your reference."
    ] * 100
    df = pd.DataFrame({'text': ham_texts, 'label': 0, 'source': 'enron_ham'})
    return df

def load_lingspam_ham():
    ham_texts = [
        "This is a legitimate email about linguistics research.",
        "Conference announcement for NLP symposium.",
        "Paper submission deadline extended.",
        "Call for papers in computational linguistics."
    ] * 50
    df = pd.DataFrame({'text': ham_texts, 'label': 0, 'source': 'lingspam_ham'})
    return df

def load_phishing_emails():
    scam_texts = [
        "Urgent: Your account will be suspended. Click here to verify.",
        "You've won a lottery! Claim your prize now.",
        "Bank alert: Unusual activity detected. Confirm identity.",
        "IRS notice: Tax refund pending. Provide details.",
        "Your package is delayed. Pay fee to expedite."
    ] * 100
    df = pd.DataFrame({'text': scam_texts, 'label': 1, 'source': 'phishing_emails'})
    return df

# Load and combine datasets
dfs = []
dfs.append(load_sms_spam())
dfs.append(load_enron_ham())
dfs.append(load_lingspam_ham())
dfs.append(load_phishing_emails())

df = pd.concat(dfs, ignore_index=True)
df = df.drop_duplicates(subset=['text'])
df['text'] = df['text'].str.lower().str.strip()

# Stratified split
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

train_df['split'] = 'train'
val_df['split'] = 'val'
test_df['split'] = 'test'

final_df = pd.concat([train_df, val_df, test_df])
final_df.to_csv(MANIFEST_PATH, index=False)
print(f"Manifest saved with {len(final_df)} samples")
print(final_df['label'].value_counts())

## 2. Training Baseline Model (TF-IDF + Logistic Regression)

In [ ]:
MODEL_DIR = "models/text/model"
os.makedirs(MODEL_DIR, exist_ok=True)

# Load data
manifest_df = pd.read_csv(MANIFEST_PATH)
train_texts = manifest_df[manifest_df['split'] == 'train']['text'].tolist()
train_labels = manifest_df[manifest_df['split'] == 'train']['label'].tolist()
val_texts = manifest_df[manifest_df['split'] == 'val']['text'].tolist()
val_labels = manifest_df[manifest_df['split'] == 'val']['label'].tolist()

# Train baseline
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train = vectorizer.fit_transform(train_texts)
X_val = vectorizer.transform(val_texts)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, train_labels)

val_preds = clf.predict_proba(X_val)[:,1]
auc = roc_auc_score(val_labels, val_preds)
val_pred_labels = clf.predict(X_val)
f1 = f1_score(val_labels, val_pred_labels)

print(f"Baseline Validation AUC: {auc:.4f}, F1: {f1:.4f}")

# Save model
joblib.dump(clf, os.path.join(MODEL_DIR, "baseline_lr.joblib"))
joblib.dump(vectorizer, os.path.join(MODEL_DIR, "tfidf_vectorizer.joblib"))
print("Baseline model saved")

## 3. Fine-tuning Transformer Model

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.softmax(torch.tensor(logits), dim=1)[:,1].numpy()
    labels = labels.numpy()
    auc = roc_auc_score(labels, preds)
    pred_labels = (preds > 0.5).astype(int)
    f1 = f1_score(labels, pred_labels)
    return {"auc": auc, "f1": f1}

# Prepare data for transformer
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

train_dataset = HFDataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})
val_dataset = HFDataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=2)

training_args = TrainingArguments(
    output_dir=MODEL_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir=os.path.join(MODEL_DIR, "logs"),
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="auc",
    greater_is_better=True,
    save_total_limit=2,
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model(MODEL_DIR)
print("Transformer model fine-tuned and saved")

## 4. Rule-Based Detection and Inference

In [ ]:
# Create rules YAML
rules_content = """
threats:
  - arrest
  - jail
  - prison
  - warrant
  - lawsuit
  - legal action
  - court
  - police
  - fbi
  - investigation

OTP:
  - otp
  - one-time password
  - verification code
  - confirm code
  - security code
  - 2fa
  - two-factor

payment:
  - pay now
  - immediate payment
  - transfer money
  - wire transfer
  - bitcoin
  - cryptocurrency
  - gift card
  - money order

urgency:
  - urgent
  - immediately
  - right now
  - time sensitive
  - deadline
  - expires soon
  - act fast
  - limited time

legalese:
  - subpoena
  - summons
  - affidavit
  - deposition
  - indictment
  - prosecution
  - compliance
  - regulatory
"""

with open("rules/text_keywords.yaml", "w") as f:
    f.write(rules_content)

print("Rules file created")

In [ ]:
class RuleBasedDetector:
    def __init__(self, rules_path: str):
        with open(rules_path, 'r') as f:
            self.rules = yaml.safe_load(f)

    def detect(self, text: str):
        text_lower = text.lower()
        matches = {}
        for category, keywords in self.rules.items():
            matches[category] = [kw for kw in keywords if kw in text_lower]
        return matches

    def score(self, text: str):
        matches = self.detect(text)
        total_matches = sum(len(v) for v in matches.values())
        return min(total_matches / 10.0, 1.0)

class BaselineModel:
    def __init__(self, model_path: str, vectorizer_path: str):
        self.clf = joblib.load(model_path)
        self.vectorizer = joblib.load(vectorizer_path)

    def predict_proba(self, texts):
        X = self.vectorizer.transform(texts)
        return self.clf.predict_proba(X)

class TransformerModel:
    def __init__(self, model_path: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.model.eval()
        self.model.to(device)

    def predict_proba(self, texts):
        encodings = self.tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
        encodings = {k: v.to(device) for k, v in encodings.items()}
        with torch.no_grad():
            outputs = self.model(**encodings)
            probs = torch.softmax(outputs.logits, dim=1).cpu().numpy()
        return probs

class TextScamDetector:
    def __init__(self, model_dir: str, rules_path: str):
        self.baseline = BaselineModel(
            os.path.join(model_dir, "baseline_lr.joblib"),
            os.path.join(model_dir, "tfidf_vectorizer.joblib")
        )
        self.transformer = TransformerModel(model_dir)
        self.rule_detector = RuleBasedDetector(rules_path)
        self.explainer = shap.Explainer(self.transformer.model, self.transformer.tokenizer)

    def predict(self, text: str):
        baseline_prob = self.baseline.predict_proba([text])[0][1]
        transformer_prob = self.transformer.predict_proba([text])[0][1]
        rule_score = self.rule_detector.score(text)
        risk = (baseline_prob + transformer_prob + rule_score) / 3.0

        rule_matches = self.rule_detector.detect(text)
        shap_values = self.explainer([text])
        top_tokens = self._get_top_tokens(shap_values, text)

        highlights = []
        for category, matches in rule_matches.items():
            for match in matches:
                highlights.append({"word": match, "category": category})

        return {
            "risk": float(risk),
            "highlights": highlights,
            "top_tokens": top_tokens,
            "details": {
                "baseline_prob": float(baseline_prob),
                "transformer_prob": float(transformer_prob),
                "rule_score": float(rule_score)
            }
        }

    def _get_top_tokens(self, shap_values, text: str):
        values = shap_values.values[0]
        tokens = shap_values.data[0]
        abs_values = np.abs(values)
        top_indices = np.argsort(abs_values)[-5:][::-1]
        return [tokens[i] for i in top_indices if tokens[i] != '[UNK]']

# Test inference
detector = TextScamDetector(MODEL_DIR, "rules/text_keywords.yaml")
result = detector.predict("Urgent: Your account will be suspended. Click here to verify.")
print("Test prediction:", result)

## 5. FastAPI Microservice

In [ ]:
app = FastAPI(title="Text Scam Detection API", description="API for detecting scam in text", version="1.0.0")

class PredictRequest(BaseModel):
    text: str

class PredictResponse(BaseModel):
    risk: float
    highlights: list
    top_tokens: list[str]

@app.get("/health")
async def health():
    return {"status": "healthy"}

@app.post("/predict/text", response_model=PredictResponse)
async def predict_text(request: PredictRequest):
    result = detector.predict(request.text)
    return PredictResponse(**result)

# For Colab, use ngrok to expose the API
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)